In [1]:
import pandas as pd
import numpy as np
# Cargar el csv en un Dataframe

path = r'C:\Users\Diego\Documents\GitHub\mitic-data-science-team-1-septiembre-2024\clases\ds-fundamentals\data\ejercicios_integracion\ventas_tienda_simulada.csv'


df= pd.read_csv(path)

df.head()


,fecha,producto,cantidad,precio_unitario,total
0,2023-01-01,Producto_A,2.0,10.5,21.0
1,2023-01-02,Producto_B,NaN,20.0,NaN
2,2023-01-03,Producto_C,5.0,NaN,NaN
3,2023-01-04,Producto_D,7.0,15.0,105.0
4,2023-01-05,Producto_A,1.0,12.0,12.0


In [2]:


#Llenar los valores faltantes con las medianas de cada columna
values = {'cantidad': df['cantidad'].median(), 'precio_unitario': df['precio_unitario'].median()}
df.fillna(values, inplace=True)


In [3]:
#Llenar la columna total con cantidad*precio_unitario
df['total'] = df.apply(
    lambda row: row['cantidad'] * row['precio_unitario'] if pd.isnull(row['total']) and not pd.isnull(row['cantidad']) and not pd.isnull(row['precio_unitario']) else row['total'], axis=1
)

In [5]:
# Estadísticas de ventas totales
print("\nEstadísticas de ventas totales:")
print(df['total'].describe())

# Resumen de ventas agrupado por producto
ventas_por_producto = df.groupby('producto')['total'].agg(
    total_ventas='sum', 
    promedio_ventas='mean', 
    mediana_ventas='median', 
    cantidad_ventas='count'
)
print("\nResumen de ventas por producto:")
print(ventas_por_producto)



Estadísticas de ventas totales:
count     20.000000
mean      69.350000
std       35.644849
min       12.000000
25%       39.750000
50%       77.500000
75%      100.000000
max      132.000000
Name: total, dtype: float64

Resumen de ventas por producto:
            total_ventas  promedio_ventas  mediana_ventas  cantidad_ventas
producto                                                                  
Producto_A         233.0             46.6            21.0                5
Producto_B         384.0             76.8            80.0                5
Producto_C         386.0             77.2            75.0                5
Producto_D         384.0             76.8            90.0                5
